# R-Multiples and Risk/Reward Analysis with Python

## Dev env configuration

In [1]:
# install the required python packages
# !pip install numpy pandas

## Imports

In [2]:
# import the necessary packages
from typing import Optional
from typing import Union
from typing import List
from dataclasses import dataclass
import pandas as pd

## Implementing a simple risk/reward calculator

In [3]:
@dataclass(frozen=True)
class TradeRiskParams:
    # define the data schema for risk/reward parameters for a given trade
    amount_to_risk: float
    percent_risk: float
    shares_to_trade: float
    total_investment: float

In [4]:
@dataclass(frozen=True)
class RLevelOutput:
    # define the data schema for R-level outputs
    r_level: int
    potential_pl: float
    price: float

In [5]:
class RiskRewardCalculator:

    # define the list of default R-values to compute
    DEFAULT_R_VALUES = [1, 2, 3, 4, 5]

    def __init__(
            self,
            total_account_value: float,
            entry_point: float,
            stop_loss: float,
            risk_rate: float = 0.01,
            is_short: bool = False
    ) -> None:
        # for short trades, the entry point must be *below* the stop loss
        if is_short and entry_point >= stop_loss:
            # raise an error
            raise ValueError(
                "For short trades, `entry_point` must be less than `stop_loss`"
            )

        # for long trades, the entry point but be *above* the stop loss
        if not is_short and entry_point <= stop_loss:
            # raise an error
            raise ValueError(
                "For long trades, `entry_point` must be greater than `stop_loss`"
            )
        
        # store the total account value, entry point, stop loss, risk rate, and
        # trade direction
        self.total_account_value = total_account_value
        self.entry_point = entry_point
        self.stop_loss = stop_loss
        self.risk_rate = risk_rate
        self.is_short = is_short

        # calculate the amount to risk on each trade
        self.amount_to_risk = self.total_account_value * self.risk_rate

        # determine the price difference between the entry point and stop loss,
        # then derive the number of shares to enter based on the amount to risk
        # and trade direction
        self.price_delta = abs(self.entry_point - self.stop_loss)
        self.shares_to_trade = self.amount_to_risk / self.price_delta

        # calculate the total amount of money to be invested based on the
        # number of shares and the entry point
        self.total_investment = self.shares_to_trade * self.entry_point

        # derive the overall risk percentage of the account
        amount_ratio = self.amount_to_risk / self.total_account_value
        self.risk_pct_of_account = amount_ratio * 100

    def get_risk_parameters(self) -> TradeRiskParams:
        # construct and return the calculated risk parameters
        return TradeRiskParams(
            amount_to_risk=self.amount_to_risk,
            percent_risk=self.risk_pct_of_account,
            shares_to_trade=self.shares_to_trade,
            total_investment=self.total_investment
        )

    def r_levels(
            self,
            r_levels: Optional[List[Union[int, float]]] = None
    ) -> List[RLevelOutput]:
        # check to see if risk/reward levels were not provided
        if not isinstance(r_levels, list):
            # use the default risk/reward levels
            r_levels = self.DEFAULT_R_VALUES

        # initialize the list of R-level outputs
        results = []

        # loop over the levels
        for r in r_levels:
            # calculate the potential profit/loss
            potential_pl = r * self.amount_to_risk

            # determine the price based on trade direction (for shorts, price
            # *decreases* as R *rises*, while for longs price *increases* as
            # R *rises)
            direction = -1 if self.is_short else 1
            price = self.entry_point + (direction * r * self.price_delta)

            # update the results list
            results.append(RLevelOutput(
                r_level=r,
                potential_pl=potential_pl,
                price=price
            ))

        # return the list of r-level results
        return results

    def r_level_at_price(self, price: float) -> float:
        # compute the r-level for the input price based on trade direction
        direction = -1 if self.is_short else 1
        r_level = direction * (price - self.entry_point) / self.price_delta
        
        # return the computed r-level for the price
        return r_level

## Using our risk/reward calculator in real-world trading situations

### Risk/reward calculation for long positions

In [6]:
# initialize the risk/reward calculator for a potential long
calc = RiskRewardCalculator(
    total_account_value=25_000,
    entry_point=100.61,
    stop_loss=95.0
)

In [7]:
# grab the risk parameters
risk_params = calc.get_risk_parameters()

# construct a dataframe from the parameters and display it
df = pd.DataFrame({
    "Amount To Risk": [risk_params.amount_to_risk],
    "Risk % of Account": [risk_params.percent_risk],
    "Shares to Trade": [risk_params.shares_to_trade],
    "Total Investment": [risk_params.total_investment],
}, index=["Trade"]).T
df

,Trade
Amount To Risk,250.000000
Risk % of Account,1.000000
Shares to Trade,44.563280
Total Investment,4483.511586


In [8]:
# calculate the risk/reward levels
r_levels = calc.r_levels()

# construct a dataframe from the R-levels and display it
df = pd.DataFrame({
    "Potential P&L": [r.potential_pl for r in r_levels],
    "Price at R": [r.price for r in r_levels],
}, index=[r.r_level for r in r_levels])
df

,Potential P&L,Price at R
1,250.0,106.22
2,500.0,111.83
3,750.0,117.44
4,1000.0,123.05
5,1250.0,128.66


In [9]:
# given an input price, determine what the R-level would be
calc.r_level_at_price(130.0)

5.238859180035651

### Risk/reward calculation for short positions

In [10]:
# initialize the risk/reward calculator for a potential short
calc = RiskRewardCalculator(
    total_account_value=25_000,
    entry_point=48.84,
    stop_loss=50.57,
    risk_rate=0.0075,
    is_short=True
)

In [11]:
# grab the risk parameters
risk_params = calc.get_risk_parameters()

# construct a dataframe from the parameters and display it
df = pd.DataFrame({
    "Amount To Risk": [risk_params.amount_to_risk],
    "Risk % of Account": [risk_params.percent_risk],
    "Shares to Trade": [risk_params.shares_to_trade],
    "Total Investment": [risk_params.total_investment],
}, index=["Trade"]).T
df

,Trade
Amount To Risk,187.500000
Risk % of Account,0.750000
Shares to Trade,108.381503
Total Investment,5293.352601


In [12]:
# calculate the risk/reward levels
r_levels = calc.r_levels()

# construct a dataframe from the R-levels and display it
df = pd.DataFrame({
    "Potential P&L": [r.potential_pl for r in r_levels],
    "Price at R": [r.price for r in r_levels],
}, index=[r.r_level for r in r_levels])
df

,Potential P&L,Price at R
1,187.5,47.11
2,375.0,45.38
3,562.5,43.65
4,750.0,41.92
5,937.5,40.19


In [13]:
# given an input price, determine what the R-level would be
calc.r_level_at_price(35.0)

8.000000000000016